#### Importing Airbnb dataset

In [709]:
# import airbnb
import pandas as pd
listings = pd.read_csv("listings.csv")
listings.dropna(axis=1)

#Dropping useless columns
listings = listings[['id', 'name','host_since',
       'host_response_time', 'host_response_rate',
       'host_is_superhost','host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'calendar_updated', 'has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'calendar_last_scraped', 'number_of_reviews',
       'number_of_reviews_ltm', 'number_of_reviews_l30d', 'first_review',
       'last_review', 'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'license', 'instant_bookable',
       'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month']]

In [710]:
#text columns to lowercase

def lowercase_strings(value):
    return value.lower() if isinstance(value, str) else value

# Apply the function to the entire DataFrame
listings = listings.applymap(lowercase_strings)

#### Read files in folder for sensor data

In [711]:
import os
import pandas as pd

def read_csv_files_in_folder(folder_path):
    # Get a list of all files in the folder
    file_list = [file for file in os.listdir(folder_path) if file.lower().endswith('.csv')]

    # Check if there are any CSV files in the folder
    if not file_list:
        raise ValueError(f"No CSV files found in the folder '{folder_path}'.")

    # Read and concatenate the CSV files
    dfs = []
    for file in file_list:
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)
        
        # Add a 'sensor' column with the name of the file
        df['sensor'] = os.path.splitext(file)[0]
        
        dfs.append(df)

    concatenated_df = pd.concat(dfs, ignore_index=True)

    return concatenated_df

In [712]:
Noise = read_csv_files_in_folder("C:\\Users\\santi\\OneDrive\\Desktop\\Flags\\Capstone\\Noise sensors")

In [713]:
Noise

,Date and Time,LAeq,sensor
0,2023-11-15 01:00:00,58.3,Ballymun
1,2023-11-15 02:00:00,57.8,Ballymun
2,2023-11-15 03:00:00,54.1,Ballymun
3,2023-11-15 04:00:00,55.0,Ballymun
4,2023-11-15 05:00:00,56.7,Ballymun
...,...,...,...
10397,2023-12-15 19:00:00,47.6,Woodstock gardens
10398,2023-12-15 20:00:00,46.3,Woodstock gardens
10399,2023-12-15 21:00:00,45.2,Woodstock gardens
10400,2023-12-15 22:00:00,45.6,Woodstock gardens


In [714]:
Air = read_csv_files_in_folder("C:\\Users\\santi\\OneDrive\\Desktop\\Flags\\Capstone\\Air sensors")

In [715]:
#Keeping only not NAN columns
Air = Air.dropna(axis=1)
Air

,Date and Time,PM<sub>10</sub>,PM<sub>2.5</sub>,sensor
0,2023-11-15 00:00:00,27.64,17.56,Ballyfermot
1,2023-11-16 00:00:00,25.57,17.41,Ballyfermot
2,2023-11-17 00:00:00,14.13,7.06,Ballyfermot
3,2023-11-18 00:00:00,8.86,5.68,Ballyfermot
4,2023-11-19 00:00:00,7.95,5.01,Ballyfermot
...,...,...,...,...
548,2023-12-11 00:00:00,7.50,6.52,Walkinstown library
549,2023-12-12 00:00:00,8.17,7.06,Walkinstown library
550,2023-12-13 00:00:00,5.34,3.80,Walkinstown library
551,2023-12-14 00:00:00,16.85,14.46,Walkinstown library


#### calculate attributes for sensors

In [716]:
Attributes_by_Sensor = Noise.groupby('sensor')['LAeq'].agg(['mean',
                                                            'max',
                                                            'min',
                                                            'var',
                                                            lambda x: (x > 60).sum(),
                                                            lambda x: (x > 70).sum(),
                                                            lambda x: (x > 80).sum()
                                                            ] )
Attributes_by_Sensor.columns = ['mean Noise',	'max Noise',	'min Noise',	'var Noise'	,'more 60 Noise',	'more 70 Noise'	,'more 80 Noise']

In [717]:
Noise['Date and Time'] = pd.to_datetime(Noise['Date and Time'])
Noise['Day name'] = Noise['Date and Time'].dt.day_name()

weekend = Noise[(Noise['Day name'] == 'Friday') | (Noise['Day name'] == 'Saturday') | (Noise['Day name'] == 'Sunday')]
weekend = weekend.groupby('sensor')['LAeq'].agg(['mean', 'max','min','var'])

new_column_names = {'mean weekend': 'max weekend', 'min weekend': 'var weekend'}

# Rename all columns
weekend.columns = ['mean weekend Noise', 'max weekend Noise', 'min weekend Noise', 'var weekend Noise']

Attributes_by_Sensor = pd.merge(weekend, Attributes_by_Sensor, on='sensor')

In [718]:
Attributes_by_Sensor_air = Air.groupby('sensor')[['PM<sub>10</sub>','PM<sub>2.5</sub>']].agg(['mean', 'max','min','var'])
Attributes_by_Sensor_air

PM<sub>10</sub>                          \
                                   mean    max   min        var   
sensor                                                            
Ballyfermot                   13.994516  27.64  3.68  47.585086   
Cabra community college        7.765806  18.69  2.70  13.989438   
Coolock                        8.952258  16.39  2.96  15.532378   
Custom house quay             13.267419  24.80  5.09  26.376166   
Davitt Road                   14.969032  29.44  3.34  63.549249   
Donnybrook fire station       10.396129  25.30  4.13  20.435691   
Dublin port                   20.784194  41.69  6.19  78.362492   
Finglas                       12.082903  27.08  3.97  34.877641   
Lord edward street             8.799677  18.30  3.43  14.190763   
Marino                        14.870323  35.17  4.98  48.611163   
Mount joy square              14.328710  29.85  4.52  42.452198   
Phoenix park                  10.280323  22.18  3.48  18.340550   
Rathmines                     15.963548  36.96  4.99  69.194004   
Ringsend                      15.158065  39.24  4.96  57.929783   
ST johns road west            15.092903  34.34  5.11  47.462008   
Sandynount green              16.050645  27.78  8.05  35.144273   
St anne's park                11.971290  22.05  4.19  23.831618   
Walkinstown library            8.114231  16.85  3.10  16.415977   

                        PM<sub>2.5</sub>                          
                                    mean    max   min        var  
sensor                                                            
Ballyfermot                     9.484839  22.11  2.48  32.097186  
Cabra community college         5.895484  12.55  2.00   8.913646  
Coolock                         7.340645  14.13  2.37  12.241040  
Custom house quay               8.881935  16.48  3.17  12.181049  
Davitt Road                     9.554194  23.10  2.30  36.068332  
Donnybrook fire station         7.457419  13.94  2.63   7.545173  
Dublin port                    10.702258  23.78  4.36  27.482978  
Finglas                         8.178065  20.59  2.60  22.297749  
Lord edward street              6.811613  13.42  2.60   9.498654  
Marino                         10.027419  28.01  3.12  34.262880  
Mount joy square                9.379355  22.24  2.41  27.868866  
Phoenix park                    6.601613  16.22  2.17  11.435394  
Rathmines                      11.029677  28.91  2.82  54.761623  
Ringsend                        8.138065  18.79  2.42  16.499936  
ST johns road west              9.565806  24.99  3.04  30.666072  
Sandynount green               12.836129  20.06  7.08  12.284045  
St anne's park                  7.789677  18.70  2.52  16.573843  
Walkinstown library             6.728462  14.46  2.43  13.641022

#### Locations of Sensors 

In [719]:
Locations_Air = pd.read_csv("C:\\Users\\santi\\OneDrive\\Desktop\\Flags\\Capstone\\locations air.csv")
Air_sensors = pd.merge(Attributes_by_Sensor_air, Locations_Air, on='sensor')

C:\Users\santi\AppData\Local\Temp\ipykernel_51088\1813605763.py:2: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left, 1 on the right)
  Air_sensors = pd.merge(Attributes_by_Sensor_air, Locations_Air, on='sensor')


In [720]:
Locations_Noise = pd.DataFrame({
    'sensor': ['Ballymun', 'Bellyfermot civic center', 'Blessington basin', 'Chancery park', 'DCC Rowing club',
               'Dolphins barn', 'Drumcondra_library', 'Mellow park', 'Navan road', 'Raheny', 'Ringsend sport centre',
               'Strand road', 'Walkinstown', 'Woodstock gardens'],
    'latitude': [53.390919, 53.343215, 53.357244, 53.34652902870364, 53.346158, 53.331141895990676, 53.36989148853905,
                 53.390391, 53.370806, 53.380108, 53.340236666389245, 53.32797988955187, 53.319599730384574, 53.32354934764496],
    'longitude': [-6.265741, -6.362101, -6.271011, -6.27222408906346, -6.320942, -6.292304521951303, -6.259054548664786,
                  -6.303545, -6.325517, -6.172811, -6.219788026210866, -6.208968500024523, -6.3221433578079855, -6.24769455032394]
})


#### Join attributes and locations for sensors

In [721]:
Noise_sensors = pd.merge(Attributes_by_Sensor, Locations_Noise, on='sensor')
Noise_sensors

,sensor,mean weekend Noise,max weekend Noise,min weekend Noise,var weekend Noise,mean Noise,max Noise,min Noise,var Noise,more 60 Noise,more 70 Noise,more 80 Noise,latitude,longitude
0,Ballymun,64.661538,72.7,56.6,10.044432,64.811709,74.8,54.1,14.408124,630,12,0,53.390919,-6.265741
1,Bellyfermot civic center,57.000321,67.2,48.4,12.645498,57.223149,80.8,46.1,18.927442,134,9,1,53.343215,-6.362101
2,Blessington basin,62.402564,86.9,42.0,107.924109,62.089367,92.0,39.9,120.772784,349,169,79,53.357244,-6.271011
3,Chancery park,60.970833,74.8,54.6,8.380851,61.140242,74.8,52.2,10.000360,505,9,0,53.346529,-6.272224
4,DCC Rowing club,55.393590,66.8,45.4,13.228448,54.900673,66.8,42.9,16.308854,35,0,0,53.346158,-6.320942
5,Dolphins barn,57.383013,71.4,47.5,14.295498,57.389367,71.4,45.9,16.613027,205,2,0,53.331142,-6.292305
6,Drumcondra_library,38.790064,48.1,32.7,8.350930,39.178466,56.9,31.9,11.401342,0,0,0,53.369891,-6.259055
7,Mellow park,56.622115,77.8,48.3,15.361085,56.959892,77.8,47.1,17.236206,146,4,0,53.390391,-6.303545
8,Navan road,55.289103,62.3,49.8,6.229334,55.021265,79.2,46.4,12.332270,29,7,0,53.370806,-6.325517
9,Raheny,54.962500,75.0,45.3,12.273412,54.969448,75.0,42.6,17.767677,35,4,0,53.380108,-6.172811


#### Function for calculate distance of each listing with sensor and assign listing to neirest sensor

In [722]:
import math

#Noise sensors
for l_index, lis in listings.iterrows():
    lx = lis['latitude']
    ly = lis['longitude']

    closeness = 100000
    closest_sensor_name = None

    for index, row in Noise_sensors.iterrows():
        nx = row['latitude']
        ny = row['longitude']
        name = row['sensor']

        distance = math.sqrt((lx - nx)**2 + (ly - ny)**2)

        if distance < closeness:
            closeness = distance
            closest_sensor_name = name

    listings.at[l_index, 'sensor Noise'] = closest_sensor_name
    
    
#AIR sensors
for l_index, lis in listings.iterrows():
    lx = lis['latitude']
    ly = lis['longitude']

    closeness = 100000
    closest_sensor_name = None

    for index, row in Air_sensors.iterrows():
        nx = row['latitude']
        ny = row['longitude']
        name = row['sensor']

        distance = math.sqrt((lx - nx)**2 + (ly - ny)**2)

        if distance < closeness:
            closeness = distance
            closest_sensor_name = name

    listings.at[l_index, 'sensor Air'] = closest_sensor_name

        



#### Merge sensor data to listings

In [723]:
listings = pd.merge(listings, Noise_sensors, left_on='sensor Noise',right_on='sensor')
listings = pd.merge(listings, Air_sensors, left_on='sensor Air',right_on='sensor')

In [724]:
listings = listings[listings['price'].notnull()]
selected_columns = listings.columns[listings.nunique() > 1]
listings = listings[selected_columns]
listings.shape

(5227, 78)

Data cleaning and transformation

In [725]:
listings.shape

(5227, 78)

In [726]:
#cjeck unique values
for x in listings.columns:
    print(x , " / unique: ", listings[x].nunique())

id  / unique:  5227
name  / unique:  4104
host_since  / unique:  2111
host_response_time  / unique:  4
host_response_rate  / unique:  54
host_is_superhost  / unique:  2
host_listings_count  / unique:  43
host_total_listings_count  / unique:  60
host_verifications  / unique:  6
host_has_profile_pic  / unique:  2
host_identity_verified  / unique:  2
latitude_x  / unique:  4605
longitude_x  / unique:  4766
property_type  / unique:  56
room_type  / unique:  4
accommodates  / unique:  16
bathrooms_text  / unique:  28
beds  / unique:  20
price  / unique:  523
minimum_nights  / unique:  53
maximum_nights  / unique:  109
minimum_minimum_nights  / unique:  51
maximum_minimum_nights  / unique:  57
minimum_maximum_nights  / unique:  100
maximum_maximum_nights  / unique:  102
minimum_nights_avg_ntm  / unique:  211
maximum_nights_avg_ntm  / unique:  269
has_availability  / unique:  2
availability_30  / unique:  31
availability_60  / unique:  61
availability_90  / unique:  91
availability_365  / uni

In [727]:
listings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5227 entries, 3 to 9019
Data columns (total 78 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            5227 non-null   int64  
 1   name                                          5227 non-null   object 
 2   host_since                                    5222 non-null   object 
 3   host_response_time                            4522 non-null   object 
 4   host_response_rate                            4522 non-null   object 
 5   host_is_superhost                             5213 non-null   object 
 6   host_listings_count                           5222 non-null   float64
 7   host_total_listings_count                     5222 non-null   float64
 8   host_verifications                            5227 non-null   object 
 9   host_has_profile_pic                          5222 non-null   o

In [728]:
for column, value in listings.iloc[2000].items():
    print(f"{column}: {value}\n   ------------- ")

id: 15771766
   ------------- 
name: home in sutton · ★4.85 · 1 bedroom · 2 beds · 1 private bath
   ------------- 
host_since: 2016-09-04
   ------------- 
host_response_time: within an hour
   ------------- 
host_response_rate: 100%
   ------------- 
host_is_superhost: t
   ------------- 
host_listings_count: 4.0
   ------------- 
host_total_listings_count: 5.0
   ------------- 
host_verifications: ['email', 'phone']
   ------------- 
host_has_profile_pic: t
   ------------- 
host_identity_verified: t
   ------------- 
latitude_x: 53.38722
   ------------- 
longitude_x: -6.14284
   ------------- 
property_type: private room in home
   ------------- 
room_type: private room
   ------------- 
accommodates: 2
   ------------- 
bathrooms_text: 1 private bath
   ------------- 
beds: 2.0
   ------------- 
price: $100.00
   ------------- 
minimum_nights: 2
   ------------- 
maximum_nights: 14
   ------------- 
minimum_minimum_nights: 2
   ------------- 
maximum_minimum_nights: 2
   --------

In [729]:
object_columns = listings.select_dtypes(include='object')
column_uniques = object_columns.nunique()
sorted_columns = column_uniques.sort_values(ascending=False)

for column, nunique in sorted_columns.items():
    print(f"{column}: {nunique}")

name: 4104
host_since: 2111
first_review: 1698
last_review: 611
price: 523
property_type: 56
host_response_rate: 54
bathrooms_text: 28
sensor Noise: 14
sensor Air: 14
sensor_x: 14
sensor_y: 14
host_verifications: 6
room_type: 4
host_response_time: 4
host_identity_verified: 2
host_has_profile_pic: 2
has_availability: 2
calendar_last_scraped: 2
host_is_superhost: 2
instant_bookable: 2


#### Bathroom cleaning

In [730]:
listings['host_response_rate'].unique()

array(['100%', nan, '89%', '99%', '56%', '40%', '96%', '70%', '60%',
       '27%', '75%', '94%', '97%', '88%', '86%', '0%', '95%', '91%',
       '57%', '67%', '93%', '50%', '90%', '80%', '92%', '83%', '98%',
       '87%', '16%', '10%', '20%', '25%', '78%', '17%', '13%', '33%',
       '63%', '73%', '47%', '71%', '43%', '22%', '14%', '77%', '82%',
       '29%', '30%', '66%', '84%', '11%', '46%', '69%', '39%', '76%',
       '38%'], dtype=object)

In [731]:
#from bathroom text to number and tyoe of bathroom
listings['bathrooms_text'] = listings['bathrooms_text'].replace('half-bath', '0.5')
listings['bathrooms_text'] = listings['bathrooms_text'].replace('shared half-bath', '0.5 shared half-bath ')
listings['bathrooms_text'] = listings['bathrooms_text'].replace('private half-bath', '0.5 half-bath ')

#number
listings['Number of bathrooms'] = listings['bathrooms_text'].str.extract('(\d+\.\d+|\d+)').astype(float)

#shared?
listings = listings[listings['bathrooms_text'].notnull()]
listings['shared bathrooms'] = listings['bathrooms_text'].str.contains('shared').astype(int)


#### Host cleaning

In [732]:
#host response rate
listings['host_response_rate'] = pd.to_numeric(listings['host_response_rate'].str.replace('%', ''))

#price
listings['price'] = listings['price'].str.replace(',', '')
listings['price'] = pd.to_numeric(listings['price'].str.replace('$', ''))
target_date = pd.to_datetime('2023-12-15')

#host since
listings['host_since'] = pd.to_datetime(listings['host_since'])
listings['host_since'] = (target_date - listings['host_since']).dt.days

#first review
listings['first_review'] = pd.to_datetime(listings['first_review'])
listings['first_review'] = (target_date - listings['first_review']).dt.days

#last review
listings['last_review'] = pd.to_datetime(listings['last_review'])
listings['last_review'] = (target_date - listings['last_review']).dt.days

C:\Users\santi\AppData\Local\Temp\ipykernel_51088\3948768787.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  listings['price'] = pd.to_numeric(listings['price'].str.replace('$', ''))


In [733]:
#remove columns
listings = listings[[                           #'id',
                                              #'name',
                                         'host_since',
                                 'host_response_time',
                                 'host_response_rate',
                                  'host_is_superhost',
                                'host_listings_count',
                          'host_total_listings_count',
                                 #'host_verifications',
                               'host_has_profile_pic',
                             'host_identity_verified',
                                         'latitude_x',
                                        'longitude_x',
                                      #'property_type',
                                          'room_type',
                                       'accommodates',
                                     #'bathrooms_text',
                                               'beds',
                                              'price',
                                     'minimum_nights',
                                     'maximum_nights',
                             'minimum_minimum_nights',
                             'maximum_minimum_nights',
                             'minimum_maximum_nights',
                             'maximum_maximum_nights',
                             'minimum_nights_avg_ntm',
                             'maximum_nights_avg_ntm',
                                   #'has_availability',
                                    'availability_30',
                                    'availability_60',
                                    'availability_90',
                                   'availability_365',
                              #'calendar_last_scraped',
                                  'number_of_reviews',
                              'number_of_reviews_ltm',
                             'number_of_reviews_l30d',
                                       'first_review',
                                        'last_review',
                               'review_scores_rating',
                             'review_scores_accuracy',
                          'review_scores_cleanliness',
                              'review_scores_checkin',
                        'review_scores_communication',
                             'review_scores_location',
                                'review_scores_value',
                                   'instant_bookable',
                     'calculated_host_listings_count',
        'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
        'calculated_host_listings_count_shared_rooms',
                                  'reviews_per_month',
                                       #'sensor Noise',
                                        # 'sensor Air',
                                         #  'sensor_x',
                                 'mean weekend Noise',
                                  'max weekend Noise',
                                  'min weekend Noise',
                                  'var weekend Noise',
                                         'mean Noise',
                                          'max Noise',
                                          'min Noise',
                                          'var Noise',
                                      'more 60 Noise',
                                      'more 70 Noise',
                                      'more 80 Noise',
                                         #'latitude_y',
                                        #'longitude_y',
                                           #'sensor_y',
                          ('PM<sub>10</sub>', 'mean'),
                           ('PM<sub>10</sub>', 'max'),
                           ('PM<sub>10</sub>', 'min'),
                           ('PM<sub>10</sub>', 'var'),
                         ('PM<sub>2.5</sub>', 'mean'),
                          ('PM<sub>2.5</sub>', 'max'),
                          ('PM<sub>2.5</sub>', 'min'),
                          ('PM<sub>2.5</sub>', 'var'),
                                           #'latitude',
                                          #'longitude',
                                'Number of bathrooms',
                                   'shared bathrooms']
    
]


In [734]:
listings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5223 entries, 3 to 9019
Data columns (total 65 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   host_since                                    5218 non-null   float64
 1   host_response_time                            4521 non-null   object 
 2   host_response_rate                            4521 non-null   float64
 3   host_is_superhost                             5209 non-null   object 
 4   host_listings_count                           5218 non-null   float64
 5   host_total_listings_count                     5218 non-null   float64
 6   host_has_profile_pic                          5218 non-null   object 
 7   host_identity_verified                        5218 non-null   object 
 8   latitude_x                                    5223 non-null   float64
 9   longitude_x                                   5223 non-null   f

In [738]:
listings.select_dtypes(include=['object']).columns

for x in ['host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 'instant_bookable']:
    listings[x] = listings[x].str.replace('t', '1')
    listings[x] = listings[x].str.replace('f', '0')
    listings[x] = pd.to_numeric(listings[x])

In [742]:
listings = pd.get_dummies(listings,columns = ['host_response_time', 'room_type'])

In [745]:
listings = listings.dropna()
listings.shape

(3754, 71)